In [9]:
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
import io
import importlib
import base64

# Skapa en uppladdningsknapp
upload_button = widgets.FileUpload(
    accept='.csv',  # Acceptera bara .csv-filer
    multiple=False  # Tillåt bara en fil att laddas upp
)

# Skapa en knapp för att köra det valda skriptet
run_button = widgets.Button(description="Run")

# Skapa en plats för att visa plotten
plot_output = widgets.Output()

# Skapa en Output-widget för att fånga utdata
out = widgets.Output()

# Skapa en HTML-widget för nedladdningslänken
download_link = widgets.HTML("")

filter_value_label = widgets.Label('Exclude values below:')

# Skapa en textwidget för att ta emot filtervärdet
filter_value = widgets.FloatText(
    value=0,  # Standardvärde
    #description='Filter values below:',  # Beskrivning som visas bredvid textfältet
)


def process_and_plot_data(button):
    global result
        
    with out:
        out.clear_output()
        # Läs in data från den uppladdade filen
        uploaded_file = upload_button.value[0]
        filename = uploaded_file['name']
        content = uploaded_file['content']
        data = pd.read_csv(io.BytesIO(content), sep=',', index_col=0)
        data.index = pd.to_datetime(data.index)
        print('First 10 rows of raw data')
        print(data.head(10))
        print('---')
        data.index.names=['time']
        data.columns=['value']
        data=data.dropna()
        data = data[data['value'] > filter_value.value]
        

        

        # Importera och kör det valda skriptet
        script_module = importlib.import_module('autoMA')
        importlib.reload(script_module)  # Tvinga omimport av modulen
        result = script_module.run(data)  # Antag att varje skript har en 'run'-funktion som tar en DataFrame som argument

        # Plotta data
        with plot_output:
            plot_output.clear_output()
            fig = go.Figure()
            data=data.dropna()
            fig.add_trace(go.Scatter(x=data.index, y=data['value'], mode='lines', name='Raw Data'))
            if len(result) != 0:
                result=result.dropna()
                fig.add_trace(go.Scatter(x=result.index, y=result['value'], mode='lines', name='Result Data'))
            fig.show()

        # Skapa nedladdningslänken
        csv = result.to_csv().encode('utf-8')
        b64 = base64.b64encode(csv)
        payload = b64.decode()
        download_link.value = f'<a download="result.csv" href="data:text/csv;base64,{payload}" target="_blank"><font size="3"><b>Download Result.csv</b></font></a>'

# Lägg till en händelsehanterare för att köra process_and_plot_data när en fil laddas upp
run_button.on_click(process_and_plot_data)



# Visa alla widgets
display(upload_button, filter_value_label, filter_value, run_button, out, plot_output, download_link)


FileUpload(value=(), accept='.csv', description='Upload')

Label(value='Exclude values below:')

FloatText(value=0.0)

Button(description='Run', style=ButtonStyle())

Output()

Output()

HTML(value='')